In [1]:
# imports
from scipy.io import loadmat
import numpy as np
import pandas as pd
from tsfresh import extract_features
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters

In [2]:
# load data
matfile_baseline = loadmat('data/raw/Ach-AT/0_0315_ach-at_0.mat')
matfile_ach = loadmat('data/raw/Ach-AT/0_0315_ach-at_1.mat')
matfile_at = loadmat('data/raw/Ach-AT/0_0315_ach-at_2.mat')

In [3]:
def make_windows(matfile, window_size = 6000):
    # retrieve the relevant data from the dictionaries
    df = pd.DataFrame(matfile['filt_data'].copy())
    df = df.T
    time = pd.DataFrame(matfile['filt_t'].copy())
    time = time.T

    # set up the time column
    time.columns = ['t']
    time['t'] = time['t'] - 60000 # make time start at 0
    time['window_id'] = time['t'] / window_size # n second windows
    time['window_id'] = time['window_id'].astype(int)

    # join the electrode signals with time
    df = pd.concat([time, df], axis=1)

    return df

In [4]:
df_base = make_windows(matfile_baseline)
df_ach = make_windows(matfile_ach)
df_at = make_windows(matfile_at)

In [7]:
# train on four electrodes - choose 4,27,34 and 57 (the centre-most ones)
df_base_4 = df_base[['t','window_id',4,27,34,57]]
df_base_4['y'] = 0
df_ach_4 = df_ach[['t','window_id',4,27,34,57]]
df_ach_4['y'] = 1
df_at_4 = df_at[['t','window_id',4,27,34,57]]
df_at_4['y'] = 2
# this could be done more efficiently, build into the function

df_4 = pd.concat([df_base_4, df_ach_4, df_at_4], axis=0, ignore_index=True)
y_4 = df_4[['window_id','y']]
df_4 = df_4.drop(columns = ['y'])
y_4.to_hdf('0_0315_4_electrodes_y.h5', key = 'y', complevel = 9)

<ipython-input-7-09db19f65da3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base_4['y'] = 0
<ipython-input-7-09db19f65da3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ach_4['y'] = 1
<ipython-input-7-09db19f65da3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [11]:
# extract minimal features and see how long it takes
X_minimal = extract_features(df_4, column_id='window_id', column_sort='t', default_fc_parameters=MinimalFCParameters())

Feature Extraction: 100%|██████████| 10/10 [00:00<00:00, 22.59it/s]


In [17]:
# extract more features
X_eff = extract_features(df_4, column_id='window_id', column_sort='t', default_fc_parameters=EfficientFCParameters())

Feature Extraction: 100%|██████████| 10/10 [11:36<00:00, 69.67s/it]


In [16]:
X_minimal.to_hdf('0_0315_4_electrodes_min.h5', key = 'features', complevel = 9)

In [19]:
X_eff.to_hdf('0_0315_4_electrodes_eff.h5',key = 'features', complevel = 9)